Code from OpenAI Cookbook
https://cookbook.openai.com/examples/chat_finetuning_data_prep

In [1]:
import pandas as pd
import os
import itertools
import json
import random
import openai
import time

import helper_functions as fu
import cookbook_function as cbf

In [2]:
os.environ['TestKey3'] = 'sk-proj-GL73kbRwhRpgN3EmXz1YT3BlbkFJEMJhTsinxQDel42BZdNz' 
client = openai.OpenAI(api_key=os.environ['TestKey3'])
headers = {
    "Authorization": f"Bearer {os.environ['TestKey3']}"
}

In [10]:
# read in ft messages for generation task
data_paths = ["data/finetuning_data/train_gen.jsonl",
              "data/finetuning_data/val_gen.jsonl",
              "data/finetuning_data/test_gen.jsonl"]

# Load the dataset
datasets=[]
for data_path in data_paths:
    cbf.validate_jsonl(data_path)
    with open(data_path, 'r', encoding='utf-8') as f:
        dataset = [json.loads(line) for line in f]
        print(type(dataset))
    datasets.append(dataset)
    # Initial dataset stats
    print("{} :".format(data_path))
    print("Num examples:", len(dataset))
    print("First example:")
    for message in dataset[0]["messages"]:
        print(message)
    

Valid jsonl file
<class 'list'>
data/finetuning_data/train_gen.jsonl :
Num examples: 20
First example:
{'role': 'system', 'content': "Imagine you are an experienced policymaker in the European Parliament. When provided with a legislative proposal supported by either a left- or right-leaning, a general or no majority, your task is to modify the text to potentially gain support from counterfactual majority. Return the revised full text of the proposal with changes highlighted. If the opposing majority would fundamentally reject the proposal based on its topic or core principles, respond with: 'A [left/right] majority would reject the proposal.'"}
{'role': 'user', 'content': 'Supporting majority: None, alter text to: Right majority. Proposal: Listing the third countries whose nationals must be in possession of visas when crossing the external borders and those whose nationals are exempt from that requirement (Kosovo). The proposed Regulation aims to transfer the reference to ‘Kosovo’ from

In [8]:
# check for format errors
for dataset in data_paths:
    cbf.check_format(dataset)

Found errors:
data_type: 36
Found errors:
data_type: 34
Found errors:
data_type: 35


In [6]:
# check for missing data, distribution of messages in each conversation,
# distribution of tokens per conversation, print token limit warnings

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for dataset in datasets:
    for ex in dataset:
        messages = ex["messages"]
        if not any(message["role"] == "system" for message in messages):
            n_missing_system += 1
        if not any(message["role"] == "user" for message in messages):
            n_missing_user += 1
        n_messages.append(len(messages))
        convo_lens.append(cbf.num_tokens_from_messages(messages))
        assistant_message_lens.append(cbf.num_assistant_tokens_from_messages(messages))
        
    print("Num examples missing system message:", n_missing_system)
    print("Num examples missing user message:", n_missing_user)
    cbf.print_distribution(n_messages, "num_messages_per_example")
    #print_distribution(convo_lens, "num_total_tokens_per_example")
    cbf.print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
    n_too_long = sum(l > 4096 for l in convo_lens)
    print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 7, 415
mean / median: 171.53846153846155, 160.0
p5 / p95: 28.200000000000017, 358.60000000000014

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [7]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(datasets[0]) # only training data
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~17888 tokens that will be charged for during training
By default, you'll train for 7 epochs on this dataset
By default, you'll be charged for ~125216 tokens


Upload validated files to OpenAI API

In [8]:
# upload validated data file to OpenAI API

train_upload = client.files.create(
  file=open("data/finetuning_data/train_gen.jsonl", "rb"),
  purpose="fine-tune"
  )
print("Uploaded training file id:", train_upload.id)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"


Uploaded training file id: file-mValQnxzwCVmTUEki1nLSsC9


Create Fine-Tuning Jobs via OpenAI Software Development Kit

In [9]:
# Process jobs
all_job_ids = cbf.process_jobs([2, 5, 10], [5, 7, 10], [1, 2], train_upload, 1)
print(all_job_ids)

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1264.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 1160, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1264.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1264.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 703, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1264.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 392, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File

['ftjob-O35IDvWOrY2NhtjjbUVgtm7b', 'ftjob-jQMdeA22Wk2ugFuaHpu1zDCv', 'ftjob-RXmyv4QmzAXytgOaWjbVtqvA', 'ftjob-ZYUSnKKMURww848wj83Ry7q5', 'ftjob-CffGWXvqX85pmysROdxFNEnr', 'ftjob-k8kBOk7P6MAIwA2l3ZlxWsNG', 'ftjob-U5qpiUGuzkrjLcDGMvpbFOJw', 'ftjob-tSFAJExQR6sQLOUmIPHq0P7b', 'ftjob-ONB7wgME4xcq2WcCvDAo1kg0', 'ftjob-4i8eGWOFdb03y5vFJyDfxlFG', 'ftjob-JdCJKM25NTGnFjGyxi32STkc', 'ftjob-NYAJ8AHtANHhX7jSQuLl7AT9']


Getting Metrics / Evaluation

In [10]:
# Extract information about the jobs
all_results = cbf.extract_job_info(all_job_ids)
display(all_results)

INFO:httpx:HTTP Request: GET https://api.openai.com/v1/fine_tuning/jobs/ftjob-O35IDvWOrY2NhtjjbUVgtm7b "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/fine_tuning/jobs/ftjob-O35IDvWOrY2NhtjjbUVgtm7b/events?limit=10 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/fine_tuning/jobs/ftjob-O35IDvWOrY2NhtjjbUVgtm7b/events?limit=10&after=ftevent-vG9KlS9Jknmgfa48ekSn96xN "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/fine_tuning/jobs/ftjob-O35IDvWOrY2NhtjjbUVgtm7b/events?limit=10&after=ftevent-JERBbPOolGrm2lvbcJZ3gqAV "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/fine_tuning/jobs/ftjob-O35IDvWOrY2NhtjjbUVgtm7b/events?limit=10&after=ftevent-cg4d7IfoXQm59ZuR8xNIipon "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.openai.com/v1/fine_tuning/jobs/ftjob-O35IDvWOrY2NhtjjbUVgtm7b/events?limit=10&after=ftevent-ZlG8QBeYpmKt553YGkq2NCf7 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://a

,job_id,learning_rate_multiplier,n_epochs,batch_size,status,event_ids,result_file_name
0,ftjob-O35IDvWOrY2NhtjjbUVgtm7b,2.0,5,1,succeeded,"[ftevent-rS0Rleo57304js0H7l5fKSCh, ftevent-02H...",[file-frvIjPnudXwxIqGic9rcVdej]
1,ftjob-jQMdeA22Wk2ugFuaHpu1zDCv,2.0,5,2,succeeded,"[ftevent-HVlq8zJGBRHHF05x4SefP1Y3, ftevent-tvA...",[file-8KLa2ki4jtXQjOiy2bwGHRGX]
2,ftjob-RXmyv4QmzAXytgOaWjbVtqvA,2.0,7,1,succeeded,"[ftevent-Ns27VkNsAL1RvFYYM0zJxd4j, ftevent-BJT...",[file-jdYpdlCqbaWQbqUT8ziyWeRU]
3,ftjob-ZYUSnKKMURww848wj83Ry7q5,2.0,7,2,succeeded,"[ftevent-1m1mfx4CSCQg9j1tFQdqjWiu, ftevent-sCX...",[file-N5zK5WCKGJz2RdmVvEtxXBhT]
4,ftjob-CffGWXvqX85pmysROdxFNEnr,2.0,10,2,succeeded,"[ftevent-ICLfRCycx19TWNpHWNkIbxIf, ftevent-rR8...",[file-6N5HJtkMxu6VM6Xk9vQPkIMJ]
5,ftjob-k8kBOk7P6MAIwA2l3ZlxWsNG,5.0,5,2,succeeded,"[ftevent-Ix51a5GUyEBIUTiIl27VYLMv, ftevent-fq9...",[file-x8V5rYzfKkFMdw3iQD1s4g6H]
6,ftjob-U5qpiUGuzkrjLcDGMvpbFOJw,5.0,7,1,succeeded,"[ftevent-WOiJZOsFnUxOSfLD4S6GM5eH, ftevent-0vv...",[file-JoiXiPPcuiKDqS3rsfe5knjj]
7,ftjob-tSFAJExQR6sQLOUmIPHq0P7b,5.0,10,1,succeeded,"[ftevent-EFFvI7GMy5sBpXc3rAQn3vNd, ftevent-S1G...",[file-ZVjWpDCZ4MyDlugZ84cfpl43]
8,ftjob-ONB7wgME4xcq2WcCvDAo1kg0,5.0,10,2,succeeded,"[ftevent-eKRKHzZkyUfr37S3QGBfpyHV, ftevent-ZAP...",[file-6TPtYrmhXqARnYh7jAgsKlNU]
9,ftjob-4i8eGWOFdb03y5vFJyDfxlFG,10.0,7,1,running,"[ftevent-K6Id4OpSlLjwNe6OKBiL4wqu, ftevent-xAE...",[]


In [11]:
# Kann weg
#one_result = extract_job_info(["ftjob-ybidDEBWMktMLknd9MZAJ4kn"])
# one_event = cbf.extract_job_info(["ftjob-ybidDEBWMktMLknd9MZAJ4kn"])

In [12]:
# Kann weg
# display(one_event)

In [13]:
# Kann weg
#display(one_event)
#result_two = get_ft_results(one_event.loc[0, "result_file_name"])
# file_id_2 = one_event.at[0, "result_file_name"]
# file_id_2 = str(file_id_2).strip()
# result_three = get_ft_results(file_id_2)
# result_three_df = pd.read_csv(result_three)
# display(result_three_df)
# result_one = get_ft_results("file-LT0GFbSE6cF3vBuCctGiQm4K")

In [18]:
# For each row in column "result_file_name", use the cell value to request
# the file from the OpenAI API using the get_ft_results function and save 
# it in a metrics dataframe 
metrics = pd.DataFrame()
checkpoints = pd.DataFrame()
all_metrics = pd.DataFrame()
all_checkpoints = pd.DataFrame()
for i, row in all_results.iterrows():
    file_id = str(row["result_file_name"]).strip()
    metrics = pd.read_csv(cbf.get_ft_results(file_id))
    # set index to job_id
    metrics["job_id"] = row["job_id"]
    metrics.set_index("job_id", inplace=True)
    all_metrics = pd.concat([all_metrics, metrics], axis=0)
    # Get checkpoints of all jobs
    checkpoints = cbf.get_checkpoint_results(row["job_id"])
    # Add all strings of chackpoints to one
    all_checkpoints = all_checkpoints + checkpoints
display(all_metrics)
print(all_checkpoints)

ERROR:cookbook_function:HTTP error occurred: 404 Client Error: Not Found for url: https://api.openai.com/v1/files/%5B'file-frvIjPnudXwxIqGic9rcVdej'%5D/content
INFO:cookbook_function:Decoded response content successfully.
ERROR:cookbook_function:HTTP error occurred: 404 Client Error: Not Found for url: https://api.openai.com/v1/files/%5B'file-8KLa2ki4jtXQjOiy2bwGHRGX'%5D/content
INFO:cookbook_function:Decoded response content successfully.
ERROR:cookbook_function:HTTP error occurred: 404 Client Error: Not Found for url: https://api.openai.com/v1/files/%5B'file-jdYpdlCqbaWQbqUT8ziyWeRU'%5D/content
INFO:cookbook_function:Decoded response content successfully.
ERROR:cookbook_function:HTTP error occurred: 404 Client Error: Not Found for url: https://api.openai.com/v1/files/%5B'file-N5zK5WCKGJz2RdmVvEtxXBhT'%5D/content
INFO:cookbook_function:Decoded response content successfully.
ERROR:cookbook_function:HTTP error occurred: 404 Client Error: Not Found for url: https://api.openai.com/v1/file

,step,train_loss,train_accuracy,valid_loss,valid_mean_token_accuracy
job_id,,,,,
ftjob-O35IDvWOrY2NhtjjbUVgtm7b,1,2.72383,0.44660,NaN,NaN
ftjob-O35IDvWOrY2NhtjjbUVgtm7b,2,1.89613,0.62360,NaN,NaN
ftjob-O35IDvWOrY2NhtjjbUVgtm7b,3,2.13064,0.58929,NaN,NaN
ftjob-O35IDvWOrY2NhtjjbUVgtm7b,4,1.86259,0.60615,NaN,NaN
ftjob-O35IDvWOrY2NhtjjbUVgtm7b,5,1.78914,0.57143,NaN,NaN
...,...,...,...,...,...
ftjob-NYAJ8AHtANHhX7jSQuLl7AT9,62,0.05830,0.97865,NaN,NaN
ftjob-NYAJ8AHtANHhX7jSQuLl7AT9,63,0.02298,0.98880,NaN,NaN
ftjob-NYAJ8AHtANHhX7jSQuLl7AT9,64,0.16298,0.95238,NaN,NaN


""


In [22]:
# Find three most minimal losses to choose the best model
min_loss = all_metrics["train_loss"].min()
min_losses = all_metrics["train_loss"].nsmallest(3)
display(min_loss)
display(min_losses)
print(f"Models {min_losses.index} are the models with the lowest losses of {min_losses.values}.")

0.01833

job_id
ftjob-O35IDvWOrY2NhtjjbUVgtm7b    0.01833
ftjob-jQMdeA22Wk2ugFuaHpu1zDCv    0.01833
ftjob-RXmyv4QmzAXytgOaWjbVtqvA    0.01833
Name: train_loss, dtype: float64

Models Index(['ftjob-O35IDvWOrY2NhtjjbUVgtm7b', 'ftjob-jQMdeA22Wk2ugFuaHpu1zDCv',
       'ftjob-RXmyv4QmzAXytgOaWjbVtqvA'],
      dtype='object', name='job_id') are the models with the lowest losses of [0.01833 0.01833 0.01833].


Using fine-tuned model via Playground or via code

In [28]:
# test_message = []
# with open("data/finetuning_data/test_gen.jsonl", "r") as f:
#     for line in f:
#         test_message.append(json.loads(line))

headers=  {'Authorization': f'Bearer sk-proj-GL73kbRwhRpgN3EmXz1YT3BlbkFJEMJhTsinxQDel42BZdNz'}

# make a request to the new model
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:lse:mig-gen:9qcUWLEC",
  messages = [{"role": "system", "content": "Imagine you are an experienced policymaker in the European Parliament. When provided with a legislative proposal supported by either a left- or right-leaning, a general or no majority, your task is to modify the text to potentially gain support from counterfactual majority. Return the revised full text of the proposal with changes highlighted. If the opposing majority would fundamentally reject the proposal based on its topic or core principles, respond with: 'A [left/right] majority would reject the proposal.'"}, 
              {"role": "user", "content": "Supporting majority: General, alter text to: right majority.  Proposal: European Border Surveillance System (EUROSUR)  PURPOSE: to establish a European Border Surveillance System (EUROSUR) with the aim of increasing coordination within and between Member States to prevent and tackle serious crime. PROPOSED ACT: Regulation of the European Parliament and of the Council. BACKGROUND: this proposal shall provide for the necessary legal framework to respond to the request of the European Council of 23-24 June 2011 to further develop the European Border Surveillance System (EUROSUR) as a matter of priority in order to become operational by 2013, allowing Member States' authorities carrying out border surveillance activities and the European Agency for the Management of Operational Cooperation at the External Borders of the Member States of the European Union (Frontex), to share operational information and improve cooperation. The aim of EUROSUR is to reinforce the control of the Schengen external borders. EUROSUR will establish a mechanism for Member States' authorities carrying out border surveillance activities to share operational information and to cooperate with each other and with the Agency in order to reduce the loss of lives at sea and the number of irregular immigrants entering the EU undetected, and to increase internal security by preventing cross-border crimes, such as trafficking in human beings and the smuggling of drugs. The works currently carried out for the testing and the gradual establishment of EUROSUR are based on a roadmap presented in a Commission Communication in 2008. IMPACT ASSESSMENT: the Commission has identified four policy options comprising of sub-options: Option 1: fully decentralised - with the support of the National Coordination Centres (NCC); Option 2: partly centralised – for the EUROSUR network; Option 3: fully centralised approach and including a sub-option 'Cooperation with third countries'; Option 4: common applications of surveillance tools at EU level. In line with the impact assessment, the following options would be the preferred ones: with regard to the establishment of NCCs, Option 1.1 is the preferred option, because it does not require Member States to restructure their national administrations and thus could be easily implemented; - following the decentralised approach for setting up EUROSUR, the preferred policy option for the EUROSUR network is Option 2.2; - taking into account the urgent need for enhancing border control in the Mediterranean region, Option 3.2 provides the best answer on how to promote the cooperation with neighbouring third countries. However, the willingness of northern African countries to cooperate is a precondition for the implementation of Option 3.2. - for the common application of surveillance tools, Option 4.2 is the option providing most added value. LEGAL BASIS: Article 77(2)(d) of the Treaty on the Functioning of the European Union (TFEU) according to which the European Parliament and the Council, acting in accordance with the ordinary legislative procedure, shall adopt measures concerning any measure necessary for the gradual establishment of an integrated management system for external borders. CONTENT: this proposal establishes a common framework for the exchange of information and cooperation between Member States and the Agency in order to improve the situational awareness and reaction capability at the external borders of the Member States and of the European Union, the European Border Surveillance System (EUROSUR). The purpose of the legislative proposal is to improve the situational awareness and reaction capability of Member States and the Agency when preventing irregular migration and cross-border crime at the external land and maritime borders. Common framework: a common framework shall be established with clear responsibilities and competencies for the national coordination centres (NCC) for border surveillance in the Member States and the Agency, which form the backbone of EUROSUR. These centres, which shall ensure an effective and efficient management of resources and personnel at national level, and the Agency shall communicate with each other via the communication network, which would allow to exchange both non-classified sensitive as well as classified information. For the exchange of information and cooperation in the field of border surveillance, Member States and the Agency shall use the framework of EUROSUR, consisting of the following components: (i) national coordination centres for border surveillance; (ii) national situational pictures; (iii) communication network; (iv) European situational picture; (v) common pre-frontier intelligence picture; (vi) common application of surveillance tools. The proposal also outlines the objectives of: Situational pictures: the cooperation and information exchange between the national coordination centres and the Agency is done via 'situational pictures', which shall be established at national and European level as well as for the pre-frontier area. These three pictures, of which the two latter shall be managed by the Agency, are structured in a similar way to facilitate the flow of information among them. The situational pictures will as a general rule not involve personal data but rather the exchange of information on incidents and depersonalised objects, such as the detection and tracking of vessels. In exceptional cases personal data may form part of the data shared by Member States with the Agency provided that the conditions of Frontex Regulation are met. To the extent personal data forms part of the national situational picture of neighbouring external border sections, it may be exchanged between neighbouring Member States only, under the conditions set by the horizontal EU legal framework on data protection; Surveillance tools: the Agency shall provide a service for the common application of surveillance tools, taking into account that such a service can be provided more cost-efficiently at European level. Such a service could be implemented with the support of relevant European space programmes, including the operational Global Monitoring for Environment and Security (GMES). The approach chosen in EUROSUR is to make best use of existing information, capabilities and systems available in other EU agencies to the extent possible. For this reason, the Agency would closely cooperate with the EU Satellite Centre, the European Fisheries Control Agency and the European Maritime Safety Agency in providing the service for the common application of surveillance tools as well as with EUROPOL in order to exchange information on cross-border crime. With regard to maritime traffic data to be provided by the SafeSeaNet system under Directive 2002/59/EC, the Commission intends to make an appropriate proposal modifying the Directive in 2013. It is envisaged that the relevant information in SafeSeaNet will also be made available for purposes other than those related to maritime safety, maritime security and marine environment protection and thereby be part of the surveillance tools used in the EUROSUR framework. Reaction capacity: better awareness of what is going on at the external borders is only of limited value if it is not complemented by an improved capability of EU Member States to react to challenges faced at their external borders. For this reason, Member States shall divide their external borders into borders sections, to which – based on risk analysis and the number of incidents occuring – impact levels shall be attributed. Depending on which impact levels have been attributed, the national coordination centres and the Agency shall take counter-measures in order to lower the impact on the border section in question. Cooperation with third countries: exisiting and planned regional networks set up between Member States and neighbouring third countries shall be linked to EUROSUR via the national coordination centres. Implementation: taking into account that Member States and the Agency are already in the process of setting up the different components of EUROSUR at national and European level, EUROSUR should become operational in the second half of 2013. The Joint Research Centre of the European Commission should provide the Agency with technical support on the further technical development of EUROSUR. Monitoring and evaluation: the Agency shall submit a report on the functioning of EUROSUR on 1 October 2015 and every two years thereafter. The Commission shall provide an overall evaluation of EUROSUR to the European Parliament and the Council on 1 October 2016 and every four years thereafter. Fundamental rights and data protection requirements: this proposal was subjected to scrutiny to ensure that its provisions are fully compatible with fundamental rights and notably human dignity, prohibition of torture and inhuman or degrading treatment or punishment, right to liberty and security, right to the protection of personal data, non-refoulement, non-discrimination and rights of the child. Particular attention was paid to Articles 4 and 19(2) of the EU Charter of Fundamental Rights, which prohibit removal of persons to a State where there is a serious risk of death penalty, torture or other inhuman or degrading treatment or punishment. The draft Regulation explicitly prohibits any exchange of information with a third country that could use this information to identify persons or groups of persons who are under a serious risk of being subjected to torture, inhuman and degrading treatment or punishment or any other violation of fundamental rights. It explicitly provides that Member States and the Agency shall give priority to the special needs of children, victims of trafficking, persons in need of urgent medical assistance, persons in need of international protection, persons in distress at sea and other persons in a particularly vulnerable situation. The protection of personal data is also of particular importance as data sharing may include personal data, in which situation the data protection rules apply and must be fully respected. Territorial provisions: this Regulation constitutes a development of provisions of the Schengen acquis, in which the United Kingdom and Ireland are not participating but which applies to 4 associated countries (Norway, Iceland, Switzerland and Liechtenstein).BUDGETARY IMPLICATION: the different components of EUROSUR will be mainly implemented by the Agency and by Member States (shared management) on the basis of the 2008 EUROSUR roadmap:- with regard to setting up the national coordination centres, Member States will be supported by the External Borders Fund in 2012-2013 and the instrument for financial support for external borders and visa as part of the planned Internal Security Fund in 2014-2020;- the Agency will use its own budget to set up the communication network and other horizontal components of EUROSUR, such as the European situational picture and the common pre-frontier intelligence picture, and when necessary this is completed by support under the Internal Security Fund (direct or indirect centralised financial management);- funding provided under the 7th Framework Programme for Research and Development will support the setting up of the envisaged service for the common application of surveillance tools in 2012-2013;- measures in neighbouring third countries will be supported in 2012-2013 by the Thematic Programme for Asylum and Migration, as part of the Development Cooperation Instrument.The impact assessment provides a financial envelope of EUR 338.7 million from 2011 to 2020."}
              ]
)
print(completion.choices[0].message)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletionMessage(content="A right majority would propose to add the additional part to Article 7, paragraph 4: 'This shall be done also to fight illegal migration and cross- border criminality, such as smuggling and trafficking in human beings, weapons and drugs; '", role='assistant', function_call=None, tool_calls=None)


In [30]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:lse:mig-gen:9qcUWLEC",
    messages = [{"role": "user", "content": "What else could a right majority change?"}
              ]
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [32]:
print(completion.choices[0].message)

ChatCompletionMessage(content='A right majority could potentially change a wide range of policies and laws, depending on the specific priorities and goals of the majority. Some examples of changes that a right majority might seek to make include:\n\n1. Tax policy: A right majority could change the tax code to reduce taxes on businesses and high-income individuals, or to simplify the tax system.\n\n2. Immigration policy: A right majority could push for stricter immigration laws, increased border security, and a crackdown on illegal immigration.\n\n3. Health care: A right majority might seek to repeal or replace the Affordable Care Act (Obamacare) with a more market-based health care system.\n\n4. Environmental regulations: A right majority could relax environmental regulations in order to reduce the burden on businesses and increase economic growth.\n\n5. Education: A right majority might seek to expand school choice options, reduce federal involvement in education, or promote alternati